In [175]:
"""
!pip install --upgrade pip
!pip install simplekml
!pip install pycollada
!pip install pandas
!pip install Pyarrow
!pip install numpy
!pip install -U ipykernel
"""
import sys
import os
import math

# os.chdir('/home/rainerth/notebooks/wind-turbine-kml/')
# os.chdir(os.path.dirname(__file__))

print(sys.path)

import collada_wt
import simplekml
# https://simplekml.readthedocs.io/en/latest/index.html

DIRECTORY_NESTING = True # Some Apps support only one level of nesting (no subfolders)

kml = simplekml.Kml()
kml.document = simplekml.Folder(name = "Windpark Bösingen")



['/home/rainerth/prj/dfvb/windpark-boesingen', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/rainerth/venv-py311/lib/python3.11/site-packages']


In [176]:
def create_kml_cylinder (kmlfolder, cyl_center, cyl_diameter = 1000, cyl_height = 100.0, cyl_color = simplekml.Color.green, cyl_name = 'cylinder', cyl_visibility = 1):

	# Mittelpunkt und Radius des Kreises
	center = cyl_center
	radius_m = cyl_diameter/2	# in m radius
	height = cyl_height

	# Umrechnung des Radius in Grad unter Berücksichtigung der Breitengrade
	radius_deg_lat = radius_m / 1000 / 111.32
	radius_deg_lon = radius_deg_lat / math.cos(math.radians(center[1]))

	# Koordinaten des Kreises
	coords = []
	for i in range(360):
		theta = math.radians(i)
		x = center[0] + radius_deg_lon * math.cos(theta)
		y = center[1] + radius_deg_lat * math.sin(theta)
		coords.append((x, y, height))

	# Polygon
	pol = kmlfolder.newpolygon(
		name=cyl_name,
		outerboundaryis=coords,
		altitudemode=simplekml.AltitudeMode.relativetoground,
		extrude=1,
		visibility=cyl_visibility
	)

	# Farbe und Transparenz des Polygons
	pol.style.linestyle.width = 1
	pol.style.linestyle.color = cyl_color
	#pol.style.linestyle.color = simplekml.Color.changealphaint(100, cyl_color)

	#pol.style.polystyle.color = cyl_color
	pol.style.polystyle.color = simplekml.Color.changealphaint(200, cyl_color)
	#pol.style.polystyle.colormode = simplekml.ColorMode.random
	pol.style.polystyle.outline = 1
	pol.style.polystyle.fill = 1


	"""
	pol.style.linestyle.color = simplekml.Color.green
	pol.style.linestyle.width = 5
	pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.green)
	"""
	return


In [177]:
# create turbine
# https://www.wind-turbine-models.com/turbines/2425-vestas-v172-7.2-enventus
# https://www.koordinaten-umrechner.de/decimal/48.237694,8.524661?karte=OpenStreetMap&zoom=16

daeurl = "models"

collada_wt1 = collada_wt.create_turbine(tower_height=199,
                    tower_bot_diameter = 6,
                    tower_top_diameter = 5,
                    nacelle_height = 3,
                    nacelle_length = 20,
                    nacelle_overhang = 8,
                    rotor_diameter = 172,
                    blade_root_length = 3.5,
                    blade_root_diameter = 3,
                    blade_chord=4,
                    blade_tip_size=0.5,
                    blade_twist=30,
                )

collada_wt1.write(daeurl + '/vestas-v172.dae')

collada_wt2 = collada_wt.create_turbine(tower_height=95,
                    tower_bot_diameter = 5,
                    tower_top_diameter = 4,
                    nacelle_height = 3,
                    nacelle_length = 10,
                    nacelle_overhang = 4,
                    rotor_diameter = 70,
                    blade_root_length = 2.5,
                    blade_root_diameter = 2,
                    blade_chord=4,
                    blade_tip_size=0.5,
                    blade_twist=30,
                )

collada_wt2.write(daeurl + '/WEA-2003.dae')


# WEA geplant
collada_zone = collada_wt.create_zone(
                    zone_height=199+172/2+100,
                    zone_diameter = 150*2+172,
                )
collada_zone.write(daeurl + '/WEA-safety-zone.dae')


# WEA 2003
collada_zone = collada_wt.create_zone(
                    zone_height=99+60/2+100,
                    zone_diameter = 150*2+60,
                )
collada_zone.write(daeurl + '/WEA-safety-zone-2003.dae')


# Testturm RW
collada_zone = collada_wt.create_zone(
                    zone_height=246,
                    zone_diameter = 24.8,
                )
collada_zone.write(daeurl + '/testturm.dae')


In [178]:
# create kml for Google Earth
# https://www.schwarzwaelder-bote.de/inhalt.windenergieanlagen-in-boesingen-drei-plus-maximal-fuenf-im-gespraech.8840b7d3-7346-4f63-8c82-27f76a697bdd.html

# read coordinates from csv

import pandas as pd
import numpy as np

def csv2df(filename):
	df = pd.read_csv(filename, sep=',', decimal='.', header=0)
	df = df.dropna(axis=0, subset=['Latitude', 'Longitude'])
	df = df[df['Latitude'] != '']
	df = df[df['Longitude'] != '']

	df['Latitude'] = df['Latitude'].astype(np.float64)
	df['Longitude'] = df['Longitude'].astype(np.float64)

	return df

def add_turbine(row, folder,
	  	folder_model,
		folder_extent,
		folder_safety,
		folder_marker,
		markerstyle, visibility=1):

	name = row['Name']
	daemodel = row['Model']
	latitude = row['Latitude']
	longitude = row['Longitude']
	height = row['height']
	diameter = row['diameter']

	# add turbine or other model - will be shown in Google Earth Pro Desktop only
	turbine = folder_model.newmodel(name=name)
	turbine.link.href = 'models/' + daemodel
	turbine.location.latitude = latitude
	turbine.location.longitude = longitude
	turbine.scale.x = 1
	turbine.scale.y = 1
	turbine.scale.z = 1
	turbine.visibility = visibility

	# add object as zylinder
	create_kml_cylinder (
		kmlfolder=folder_extent,
		cyl_center=(longitude, latitude),
		cyl_diameter=diameter,
		cyl_height=height + diameter/2,
		cyl_color=simplekml.Color.firebrick,
		cyl_name=name,
		cyl_visibility=visibility
	)

	# add safety zone as zylinder
	if folder_safety is not None:
		create_kml_cylinder (
			kmlfolder=folder_safety,
			cyl_center=(longitude, latitude),
			cyl_diameter=diameter + 150*2,
			cyl_height=height + diameter/2 + 100,
			cyl_color=simplekml.Color.lightgray,
			cyl_name=name,
			cyl_visibility=visibility
		)

	# add marker for turbine
	if folder_marker is not None and markerstyle is not None:
		location = folder_marker.newpoint(name=name, coords=[(longitude, latitude)])
		location.style=markerstyle
		location.visibility = visibility

	return

# simple WEA locations with icon
markerstyle = simplekml.Style()
markerstyle.labelstyle.color = simplekml.Color.lightgray  # Make the text red
markerstyle.labelstyle.scale = 0.8  # Make the text twice as big
markerstyle.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'
markerstyle.iconstyle.color = simplekml.Color.darkgray

markerstyle_badenova = simplekml.Style()
markerstyle_badenova.labelstyle.color = simplekml.Color.lightgray  # Make the text blue
markerstyle_badenova.labelstyle.scale = 0.8  # Make the text twice as big
markerstyle_badenova.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'
markerstyle_badenova.iconstyle.color = simplekml.Color.lightblue

markerstyle_iterra = simplekml.Style()
markerstyle_iterra.labelstyle.color = simplekml.Color.lightgray  # Make the text red
markerstyle_iterra.labelstyle.scale = 0.8  # Make the text twice as big
markerstyle_iterra.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'
markerstyle_iterra.iconstyle.color = simplekml.Color.red


# Badenova
folder_name = 'Badenova'
if DIRECTORY_NESTING:
	folder = kml.newfolder(name=folder_name)
	folder.visibility = 1

folder_model = folder.newfolder(name='Modell') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Modell')
folder_model.visibility = 1

folder_extent = folder.newfolder(name='Umfang') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Umfang')
folder_extent.visibility = 1

folder_safety =	folder.newfolder(name='Sicherheitsabstand') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Sicherheitsabstand')
folder_safety.visibility = 0

folder_marker = folder.newfolder(name='Standorte') if DIRECTORY_NESTING \
	else  kml.newfolder(name=folder_name + '-' + 'Standorte')
folder_marker.visibility = 0

df = csv2df('locations/badenova.csv')
df.apply(lambda row: add_turbine(row, folder,
	folder_model, folder_extent, folder_safety, folder_marker,
	markerstyle_badenova, visibility=1), axis=1
)

# iTerra
folder_name = 'iTerra'
if DIRECTORY_NESTING:
	folder = kml.newfolder(name=folder_name)
	folder.visibility = 1

folder_model = folder.newfolder(name='Modell') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Modell')
folder_model.visibility = 1

folder_extent = folder.newfolder(name='Umfang') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Umfang')
folder_extent.visibility = 1

folder_safety =	folder.newfolder(name='Sicherheitsabstand') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Sicherheitsabstand')
folder_safety.visibility = 0

folder_marker = folder.newfolder(name='Standorte') if DIRECTORY_NESTING \
	else  kml.newfolder(name=folder_name + '-' + 'Standorte')
folder_marker.visibility = 0

df = csv2df('locations/iterra.csv')
df.apply(lambda row: add_turbine(row, folder,
	folder_model, folder_extent, folder_safety, folder_marker,
	markerstyle_iterra, visibility=1), axis=1
)

# Dunningen
folder_name = 'Dunningen'
if DIRECTORY_NESTING:
	folder = kml.newfolder(name=folder_name)
	folder.visibility = 1

folder_model = folder.newfolder(name='Modell') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Modell')
folder_model.visibility = 1

folder_extent = folder.newfolder(name='Umfang') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Umfang')
folder_extent.visibility = 1

folder_safety =	folder.newfolder(name='Sicherheitsabstand') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Sicherheitsabstand')
folder_safety.visibility = 0

folder_marker = folder.newfolder(name='Standorte') if DIRECTORY_NESTING \
	else  kml.newfolder(name=folder_name + '-' + 'Standorte')
folder_marker.visibility = 0

df = csv2df('locations/dunningen.csv')
df.apply(lambda row: add_turbine(row, folder,
	folder_model, folder_extent, folder_safety, folder_marker,
	markerstyle, visibility=1), axis=1
)

# Herrenzimmern
folder_name = 'Herrenzimmern'
if DIRECTORY_NESTING:
	folder = kml.newfolder(name=folder_name)
	folder.visibility = 1

folder_model = folder.newfolder(name='Modell') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Modell')
folder_model.visibility = 1

folder_extent = folder.newfolder(name='Umfang') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Umfang')
folder_extent.visibility = 1

folder_safety =	folder.newfolder(name='Sicherheitsabstand') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Sicherheitsabstand')
folder_safety.visibility = 0

folder_marker = folder.newfolder(name='Standorte')
folder_marker.visibility = 0

df = csv2df('locations/herrenzimmern.csv')
df.apply(lambda row: add_turbine(row, folder,
	folder_model, folder_extent, folder_safety, folder_marker,
	markerstyle, visibility=1), axis=1
)

# Testturm
folder_name = 'Testturm'
if DIRECTORY_NESTING:
	folder = kml.newfolder(name=folder_name)
	folder.visibility = 1

folder_model = folder.newfolder(name='Modell') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Modell')
folder_model.visibility = 1

folder_extent = folder.newfolder(name='Umfang') if DIRECTORY_NESTING \
	else kml.newfolder(name=folder_name + '-' + 'Umfang')
folder_extent.visibility = 1

folder_safety =	None
folder_marker = None

df = csv2df('locations/testturm.csv')
df.apply(lambda row: add_turbine(row, folder,
	folder_model, folder_extent, folder_safety, folder_marker,
	None, visibility=1), axis=1
)


0    None
1    None
dtype: object

In [179]:
import math
import simplekml

# Erstellen Sie ein neues KML-Objekt
# kml = simplekml.Kml()
folder = kml.newfolder(name='Flugplatz')
airfield_boesingen = (8.534621455251553, 48.227697118500274)


#### Landebahn Bösingen

coordinates_text = """
8.531799515709411,48.22788047237495,0
8.531791237373254,48.22746570381057,0
8.537614083339131,48.22746248646123,0
8.537607267242425,48.22789309924998,0
8.531799515709411,48.22788047237495,0
"""

lines = coordinates_text.strip().split('\n')
coordinates = [tuple(map(float, line.split(','))) for line in lines]

pol = folder.newpolygon(
	name="Landebahn",
	outerboundaryis=coordinates,
	altitudemode=simplekml.AltitudeMode.clamptoground
)
pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.blue)


#### Schutzzone Bösingen

# das Zentrum und den Radius des Kreises
center = airfield_boesingen
radius = 2000 / 111139  # Umwandlung von Metern in Grad

# Berechnen der Koordinaten des Kreises
coords = []
for i in range(360):
    theta = math.radians(i)
    x = center[0] + radius * math.cos(theta) / math.cos(math.radians(center[1]))  # Anpassung für Längenverzerrung
    y = center[1] + radius * math.sin(theta)
    coords.append((x, y, 0))  # Höhe 0, um den Kreis an die Oberfläche zu binden

#  Polygon, das den Kreis darstellt
pol = folder.newpolygon(
    name="Schutzzone",
    outerboundaryis=coords,
    altitudemode=simplekml.AltitudeMode.clamptoground
)
# Farbe des Polygons auf Rot und transparent setzen
pol.style.polystyle.color = simplekml.Color.changealphaint(0, simplekml.Color.red)

# Linienfarbe auf Rot und die Linienbreite auf 5 setzen
pol.style.linestyle.color = simplekml.Color.red
pol.style.linestyle.width = 2


#### Endanflugbereich Bösingen

radius_m = 5000	# in Koordinaten-Einheiten (Grad)
height = 500.0

create_kml_cylinder (
	kmlfolder=folder,
    cyl_center=airfield_boesingen,
    cyl_diameter=radius_m * 2,
    cyl_height=height,
    cyl_color=simplekml.Color.green,
    cyl_name='Endanflugbereich',
	cyl_visibility=0
)



In [180]:
# Struktur mit Koordinaten: (Längengrad, Breitengrad, Höhe)
folder = kml.newfolder(name='Windvorranggebiete')

# Windvorranggebiet 1
coordinates_text = """
8.538625953140679,48.24733108376624,0
8.545398800655983,48.25155668096118,0
8.545061726012847,48.255504959821,0
8.538271166860046,48.25418519218096,0
8.521300387225741,48.25642433277036,0
8.519359663435308,48.24964241078206,0
8.523113634981584,48.25018720725038,0
8.524434812536384,48.2480791750467,0
8.521744597138214,48.24594686097112,0
8.522702190206415,48.24477797568368,0
8.525950765261815,48.2442144809804,0
8.53804803344558,48.24392246020017,0
8.538625953140679,48.24733108376624,0
"""

lines = coordinates_text.strip().split('\n')
coordinates = [tuple(map(float, line.split(','))) for line in lines]

# Aus den Koordinaten ein Polygon mit gelber Füllung und roter Linie erstellen
pol = folder.newpolygon(
	name="Windvorranggebiet 1",
	outerboundaryis=coordinates,
	altitudemode=simplekml.AltitudeMode.clamptoground
)
pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.yellow)
pol.style.linestyle.color = simplekml.Color.red
pol.style.linestyle.width = 2
pol.description = "Windvorranggebiet 1"


# Windvorranggebiet 2
coordinates_text = """
8.514070913418958,48.26142175265778,0
8.514622479179902,48.26552246326965,0
8.527729663701306,48.26672900175677,0
8.531674441607455,48.26384212287596,0
8.532523110802014,48.26421272078534,0
8.530014534666856,48.26701405884985,0
8.525004254661457,48.27837712280289,0
8.520348560492131,48.27584428565216,0
8.512373126115014,48.27177096214482,0
8.506707838964472,48.26880690993875,0
8.507707710669782,48.2564529695051,0
8.517445889736155,48.25661666038091,0
8.514070913418958,48.26142175265778,0
"""

# Teilen Sie den Text in eine Liste von Zeilen
lines = coordinates_text.strip().split('\n')

# Wandeln Sie jede Zeile in ein Tupel von Zahlen um
coordinates = [tuple(map(float, line.split(','))) for line in lines]

# Aus den Koordinaten ein Polygon mit gelber Füllung und roter Linie erstellen
pol = folder.newpolygon(
	name="Windvorranggebiet 2",
	outerboundaryis=coordinates,
	altitudemode=simplekml.AltitudeMode.clamptoground
)
pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.yellow)
pol.style.linestyle.color = simplekml.Color.red
pol.style.linestyle.width = 2
pol.description = "Windvorranggebiet 2"

# Windvorranggebiet 3
coordinates_text = """
8.5522629907101,48.26321448744435,0
8.552258376763195,48.25989643569763,0
8.553128098300105,48.25368698104617,0
8.558522604953403,48.25027521335589,0
8.561505433168408,48.25019772319624,0
8.564563501824072,48.25381918640282,0
8.564232916284588,48.25828634025782,0
8.559161166027341,48.26434588502315,0
8.559146231551225,48.26700202858952,0
8.554157230908018,48.26591884900093,0
8.554485660548242,48.27367261632118,0
8.548681751883251,48.27471567553113,0
8.547827630360242,48.27003625407925,0
8.5522629907101,48.26321448744435,0
"""

lines = coordinates_text.strip().split('\n')
coordinates = [tuple(map(float, line.split(','))) for line in lines]

# Aus den Koordinaten ein Polygon mit gelber Füllung und roter Linie erstellen
pol = folder.newpolygon(
	name="Windvorranggebiet 3",
	outerboundaryis=coordinates,
	altitudemode=simplekml.AltitudeMode.clamptoground
)
pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.yellow)
pol.style.linestyle.color = simplekml.Color.red
pol.style.linestyle.width = 2
pol.description = "Windvorranggebiet 3"


# Windvorranggebiet 4
coordinates_text = """
8.566613951316528,48.21751577350571,0
8.563450873050797,48.21486175121698,0
8.552957993518984,48.20980641549801,0
8.556186048885095,48.20665602637702,0
8.561204273953242,48.20732363941927,0
8.565200989995743,48.20424968308848,0
8.569392397898296,48.20868686765963,0
8.574446120720101,48.21026664619694,0
8.566613951316528,48.21751577350571,0
"""

lines = coordinates_text.strip().split('\n')
coordinates = [tuple(map(float, line.split(','))) for line in lines]

# Aus den Koordinaten ein Polygon mit gelber Füllung und roter Linie erstellen
pol = folder.newpolygon(
	name="Windvorranggebiet 4",
	outerboundaryis=coordinates,
	altitudemode=simplekml.AltitudeMode.clamptoground
)
pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.yellow)
pol.style.linestyle.color = simplekml.Color.red
pol.style.linestyle.width = 2
pol.description = "Windvorranggebiet 4"



In [181]:
def create_kml_slope(kml_folder, start_lat, start_lon, start_altitude, direction, length, slope, width, color=simplekml.Color.yellow, name="slope"):
    import math
    import simplekml

    transparency = 120

    # Umrechnung von Grad in Radiant
    direction_rad = math.radians(direction)
    slope_rad = math.radians(slope)

    # Berechnung der Höhe und der horizontalen Entfernung
    height = length * math.sin(slope_rad)  # in Kilometern
    distance = length * math.cos(slope_rad)  # in Kilometern

    # Verschiebung um die halbe Breite
    half_width = width / 2
    start_lat -= half_width * math.cos(direction_rad + math.pi / 2) / 111.32  # Umrechnung von Kilometern in Grad
    start_lon -= half_width * math.sin(direction_rad + math.pi / 2) / (111.32 * math.cos(math.radians(start_lat)))  # Umrechnung von Kilometern in Grad

    # Berechnung der Endkoordinaten
    end_lat = start_lat + distance * math.cos(direction_rad) / 111.32  # Umrechnung von Kilometern in Grad
    end_lon = start_lon + distance * math.sin(direction_rad) / (111.32 * math.cos(math.radians(start_lat)))  # Umrechnung von Kilometern in Grad
    end_altitude = start_altitude + height * 1000  # Umrechnung von Kilometern in Metern

    # Berechnung der Breitenkoordinaten
    width_lat1 = start_lat + width * math.cos(direction_rad + math.pi / 2) / 111.32  # Umrechnung von Kilometern in Grad
    width_lon1 = start_lon + width * math.sin(direction_rad + math.pi / 2) / (111.32 * math.cos(math.radians(start_lat)))  # Umrechnung von Kilometern in Grad

    width_lat2 = end_lat + width * math.cos(direction_rad + math.pi / 2) / 111.32  # Umrechnung von Kilometern in Grad
    width_lon2 = end_lon + width * math.sin(direction_rad + math.pi / 2) / (111.32 * math.cos(math.radians(end_lat)))  # Umrechnung von Kilometern in Grad

    # Erstellen Sie die Eckpunkte der Fläche
    coordinates = [
        (start_lon, start_lat, start_altitude),
        (end_lon, end_lat, end_altitude),
        (width_lon2, width_lat2, end_altitude),
        (width_lon1, width_lat1, start_altitude),
    ]

    # Fügen Sie das Polygon zum KML-Ordner hinzu
    pol = kml_folder.newpolygon(name=name, outerboundaryis=coordinates, altitudemode=simplekml.AltitudeMode.absolute)
    pol.style.polystyle.color = simplekml.Color.changealphaint(transparency, color)
    pol.style.linestyle.color = simplekml.Color.changealphaint(transparency, color)


In [182]:
def create_kml_cube(kml_folder, start_lat, start_lon, start_altitude, direction, length, slope, width, color=simplekml.Color.yellow, name="cube"):

	import math
	import simplekml

	cubehight = 100
	transparency = 80

	# Umrechnung von Grad in Radiant
	direction_rad = math.radians(direction)
	slope_rad = math.radians(slope)

	# Berechnung der Höhe und der horizontalen Entfernung
	height = length * math.sin(slope_rad)  # in Kilometern
	distance = length * math.cos(slope_rad)  # in Kilometern

	# Verschiebung um die halbe Breite
	half_width = width / 2
	start_lat -= half_width * math.cos(direction_rad + math.pi / 2) / 111.32  # Umrechnung von Kilometern in Grad
	start_lon -= half_width * math.sin(direction_rad + math.pi / 2) / (111.32 * math.cos(math.radians(start_lat)))  # Umrechnung von Kilometern in Grad

	# Berechnung der Endkoordinaten
	end_lat = start_lat + distance * math.cos(direction_rad) / 111.32  # Umrechnung von Kilometern in Grad
	end_lon = start_lon + distance * math.sin(direction_rad) / (111.32 * math.cos(math.radians(start_lat)))  # Umrechnung von Kilometern in Grad
	end_altitude = start_altitude + height * 1000  # Umrechnung von Kilometern in Metern

	# Berechnung der Breitenkoordinaten
	width_lat1 = start_lat + width * math.cos(direction_rad + math.pi / 2) / 111.32  # Umrechnung von Kilometern in Grad
	width_lon1 = start_lon + width * math.sin(direction_rad + math.pi / 2) / (111.32 * math.cos(math.radians(start_lat)))  # Umrechnung von Kilometern in Grad

	width_lat2 = end_lat + width * math.cos(direction_rad + math.pi / 2) / 111.32  # Umrechnung von Kilometern in Grad
	width_lon2 = end_lon + width * math.sin(direction_rad + math.pi / 2) / (111.32 * math.cos(math.radians(end_lat)))  # Umrechnung von Kilometern in Grad

	# Erstellen Sie die Eckpunkte der unteren und oberen Fläche
	lower_coordinates = [
		(start_lon, start_lat, start_altitude),
		(end_lon, end_lat, end_altitude),
		(width_lon2, width_lat2, end_altitude),
		(width_lon1, width_lat1, start_altitude),
	]

	upper_coordinates = [
		(start_lon, start_lat, start_altitude + cubehight),
		(end_lon, end_lat, end_altitude + cubehight),
		(width_lon2, width_lat2, end_altitude + cubehight),
		(width_lon1, width_lat1, start_altitude + cubehight),
	]

	# Erstellen Sie die Eckpunkte der seitlichen Flächen
	side1_coordinates = [
		lower_coordinates[0],
		upper_coordinates[0],
		upper_coordinates[1],
		lower_coordinates[1],
	]

	side2_coordinates = [
		lower_coordinates[1],
		upper_coordinates[1],
		upper_coordinates[2],
		lower_coordinates[2],
	]

	side3_coordinates = [
		lower_coordinates[2],
		upper_coordinates[2],
		upper_coordinates[3],
		lower_coordinates[3],
	]

	side4_coordinates = [
		lower_coordinates[3],
		upper_coordinates[3],
		upper_coordinates[0],
		lower_coordinates[0],
	]

	# Fügen Sie die Polygone zum KML-Ordner hinzu
	lower_pol = kml_folder.newpolygon(name=name, outerboundaryis=lower_coordinates, altitudemode=simplekml.AltitudeMode.absolute)
	lower_pol.style.polystyle.color = simplekml.Color.changealphaint(transparency, color)
	lower_pol.linestyle.color = simplekml.Color.changealphaint(transparency, color)

	upper_pol = kml_folder.newpolygon(name=name, outerboundaryis=upper_coordinates, altitudemode=simplekml.AltitudeMode.absolute)
	upper_pol.style.polystyle.color = simplekml.Color.changealphaint(transparency, color)
	upper_pol.linestyle.color = simplekml.Color.changealphaint(transparency, color)

	side1_pol = kml_folder.newpolygon(name=name, outerboundaryis=side1_coordinates, altitudemode=simplekml.AltitudeMode.absolute)
	side1_pol.style.polystyle.color = simplekml.Color.changealphaint(transparency, color)
	side1_pol.linestyle.color = simplekml.Color.changealphaint(transparency, color)

	side2_pol = kml_folder.newpolygon(name=name, outerboundaryis=side2_coordinates, altitudemode=simplekml.AltitudeMode.absolute)
	side2_pol.style.polystyle.color = simplekml.Color.changealphaint(transparency, color)
	side2_pol.linestyle.color = simplekml.Color.changealphaint(transparency, color)

	side3_pol = kml_folder.newpolygon(name=name, outerboundaryis=side3_coordinates, altitudemode=simplekml.AltitudeMode.absolute)
	side3_pol.style.polystyle.color = simplekml.Color.changealphaint(transparency, color)
	side3_pol.linestyle.color = simplekml.Color.changealphaint(transparency, color)

	side4_pol = kml_folder.newpolygon(name=name, outerboundaryis=side4_coordinates, altitudemode=simplekml.AltitudeMode.absolute)
	side4_pol.style.polystyle.color = simplekml.Color.changealphaint(transparency, color)
	side4_pol.linestyle.color = simplekml.Color.changealphaint(transparency, color)

	return


In [183]:

def calculate_slope(speed_kmh, climb_rate_ms):
    # Umwandlung der Geschwindigkeit von km/h in m/s
    speed_ms = speed_kmh * 1000 / 3600

    # Berechnung der Steigung
    slope_rad = math.atan(climb_rate_ms / speed_ms)

    # Umwandlung der Steigung von Radiant in Grad
    slope_deg = math.degrees(slope_rad)

    return slope_deg


In [184]:
#### Abflug Bösingen

folder = kml.newfolder(name='Abflug')

# Startkoordinaten
lon, lat = airfield_boesingen

# typischer Anflug Bösingen
start_altitude = 700  # in Metern
slope = calculate_slope(60,1)	# 60 km/h bei 1 m/s
length = 3  # Länge des Quaders in Kilometern
width = 0.3  # Breite des Quaders in Kilometern
direction = 270
create_kml_slope(folder, lat, lon, start_altitude, direction, length, slope, width, simplekml.Color.yellow, "Anflug 27")

direction = 90
create_kml_slope(folder, lat, lon, start_altitude, direction, length, slope, width, simplekml.Color.yellow, "Anflug 09")

# typischer Abflug im Schleppverbund
start_altitude = 700  # in Metern
slope = calculate_slope(60,1.5)	# 60 km/h bei 1.5 m/s
length = 5  # Länge des Quaders in Kilometern
width = 2  # Breite des Quaders in Kilometern
direction = 0
create_kml_cube(folder, lat, lon, start_altitude, direction, length, slope, width, simplekml.Color.blue, "Abflug")



In [185]:
# add return range to kml
# import rthkml

# Mittelpunkt und Radius des Kreises
radius_m = 5000	# in Koordinaten-Einheiten (Grad)
height = 500.0

create_kml_cylinder (
	kmlfolder=folder,
    cyl_center=airfield_boesingen,
    cyl_diameter=radius_m * 2,
    cyl_height=height,
    cyl_color=simplekml.Color.green,
    cyl_name='Endanflugbereich',
	cyl_visibility=0
)



In [186]:
# save models to kml

kml_file = './output/WEA-boesingen.kml'
kml.save(kml_file)

In [187]:
# create kmz from kml and dae files

import zipfile

def create_kmz(kml_filename, dae_filenames, output_filename):
    with zipfile.ZipFile(output_filename, 'w') as kmz:
        kmz.write(kml_filename)

        for dae in dae_filenames:
            kmz.write(dae, os.path.join("models", os.path.basename(dae)))

# Listet alle .dae-Dateien im "model"-Verzeichnis auf

dae_files = [os.path.join("models", f) for f in os.listdir("models") if f.endswith('.dae')]
kml_file = "./output/WEA-boesingen.kml"  # Ersetzen Sie dies durch den Pfad zu Ihrer KML-Datei
output_file = "./output/WEA-boesingen.kmz"
create_kmz(kml_file, dae_files, output_file)


In [188]:


import simplekml
kml = simplekml.Kml()
pol = kml.newpolygon(name='A Polygon')
pol.outerboundaryis = [(18.333868,-34.038274), (18.370618,-34.034421),
                       (18.350616,-34.051677),(18.333868,-34.038274)]
pol.innerboundaryis = [(18.347171,-34.040177), (18.355741,-34.039730),
                       (18.350467,-34.048388),(18.347171,-34.040177)]
pol.style.linestyle.color = simplekml.Color.green
pol.style.linestyle.width = 5
pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.green)
kml.save("Polygon Styling.kml")



## Quellen

* https://www.landesrecht-bw.de/jportal/?quelle=jlink&docid=JURE060017391&psml=bsbawueprod.psml&max=true&doc.part=L&doc.norm=all
* https://www.daec.de/media/files/2022/Fachbereiche/Umwelt/Leitfaden_Luftfahrthindernisse_14_03_2022.pdf
* https://www.fsco.de/images/Dokumente/Studie_DLR.pdf
* https://www.dhv.de/fileadmin/user_upload/aktuell_zu_halten/Gelaende/DHV_info_199_windkraft.pdf
* http://www.pontepress.de/pdf/u12_201706.pdf
